# 단어의 표현
-기계는 문자를 그대로 인식할 수 없기 때문에 숫자로 변환

# 1. 원-핫 인코딩 (One-Hot Encoding)

## 1.1 직접 구현해보기

In [1]:
word_ls = ['원숭이','바나나','사과','사과']

In [2]:
from collections import defaultdict
import numpy as np 


In [10]:
def one_hot_encode(word_ls):
  #고유 단어와 인덱스를 매칭시켜주는사전 생성
  word2id_dic = defaultdict(lambda:len(word2id_dic))

  # {단어 : 인덱스 } 사전 구축
  for word in word_ls:
    word2id_dic[word]
  
  n_unique_words = len(word2id_dic) #고유한 단어의 개수
  one_hot_vectors=np.zeros((len(word_ls),n_unique_words))

  for i,word in enumerate(word_ls):
    index=word2id_dic[word] #해당 단어의 고유 인덱스
    one_hot_vectors[i,index]=1 #해당 단어의 고유 인덱스에만 1을 더해줌

  return one_hot_vectors


In [11]:
one_hot_vectors=one_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [12]:
# 코끼리 추가
word_ls = ['원숭이','바나나','사과','코끼리']

one_hot_vectors = one_hot_encode(word_ls)
one_hot_vectors

array([[1., 0., 0., 0.],
       [0., 1., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 1.]])

## 3. sklearn 활용

In [13]:
from numpy import array
from numpy import argmax 
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder

In [14]:
values=array(word_ls)
print(values)

['원숭이' '바나나' '사과' '코끼리']


In [15]:
#문자열에 숫자를 붙임
label_enc =LabelEncoder()
int_enc=label_enc.fit_transform(values)
print(int_enc)

[2 0 1 3]


In [16]:
#binary encode
onehot_enc=OneHotEncoder(sparse=False)
int_enc=int_enc.reshape(len(int_enc),1) #n:1 matrix 변환
print(int_enc)
onehot_enc=onehot_enc.fit_transform(int_enc)
print(onehot_enc)

[[2]
 [0]
 [1]
 [3]]
[[0. 0. 1. 0.]
 [1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]]


In [17]:
#one-hot encoding 의 첫번째 배열 값을 역으로 산출

In [18]:
inverted=label_enc.inverse_transform([argmax(onehot_enc[0, :])])
print(inverted)

['원숭이']


In [19]:
onehot_enc[0,:]

array([0., 0., 1., 0.])

In [20]:
argmax(onehot_enc[0, :])

2

In [21]:
label_enc.inverse_transform([argmax(onehot_enc[0,:])])

array(['원숭이'], dtype='<U3')

# 2. 밀집벡터 (Dense Vector)

In [22]:
word_embedding_dic={
    '사과' : [1.0, 0.5],
    '바나나' : [0.9, 1.2],
    '원숭이' : [0.5, 1.5]
}

## 2-1 유사도 계산

### 2.1.1. 유클리디안 거리(Euclidean distance)

In [23]:
import numpy as np 
def euclidean_dist(x,y):
  x=np.array(x)
  y=np.array(y)
  return np.sqrt(np.sum(x-y)**2)

In [24]:
euclidean_dist(word_embedding_dic['사과'],word_embedding_dic['바나나'])

0.6

### 2.1.2. 코사인 유사도(Cosine Similarity)
- 두 벡터간의 유사도를 측정하는 방법 중 하나
- 두 벡터 사이의 코사인을 측정
- 0도=1, 90도=0, 180도=-1 ==> 1에 가까울 수록 유사도가 높음

In [25]:
def cosine_similarity(x,y):
  nominator=np.dot(x,y)
  denominator=np.linalg.norm(x)*np.linalg.norm(y)
  return nominator/denominator

In [26]:
a=np.array([1,2])
b=np.array([3,4])
np.dot(a,b)

11

In [27]:
#numpy의 linalg 서브 패키지의 norm 명령으로 벡터의 길이를 계산할 수 있다. 
#위에서 예로 든 2차원 벡터 𝑎=[1,2] 의 길이는 √5≈2.236 이다.
a=np.array([1,2])
np.linalg.norm(a)

2.23606797749979

In [28]:
#사과와 바나나의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['바나나']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['바나나']))
#
# 사과와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['사과'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['사과'], word_embedding_dic['원숭이']))
# 바나나와 원숭이의 코사인 유사도
print(cosine_similarity(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))
print(euclidean_dist(word_embedding_dic['바나나'], word_embedding_dic['원숭이']))


0.8944271909999159
0.6
0.7071067811865475
0.5
0.9486832980505138
0.09999999999999998


### 2.1.3. 자카드 유사도(Jaccard index)

In [29]:
s1 = '대부분 원숭이는 바나나를 좋아합니다.'
s2 = '코주부 원숭이는 바나나를 싫어합니다.'


In [30]:
#토큰화를 수행
token_s1=s1.split()
token_s2=s2.split()

union=set(token_s1).union(set(token_s2))
print(union)

{'원숭이는', '코주부', '싫어합니다.', '대부분', '바나나를', '좋아합니다.'}


In [31]:
intersection =set(token_s1).intersection(set(token_s2))
print(intersection)

{'바나나를', '원숭이는'}


In [32]:
print(len(intersection)/len(union))

0.3333333333333333


## 3. TF-IDF를 활용한 단어 벡터

In [33]:
d1 = 'The cat sat on my face I hate a cat'
d2 = 'The dog sat on my bed I love a dog'

In [34]:
from math import log10

In [35]:
#document 내 토큰이 등장한 빈도수 계산
def f(t,d):
  return d.count(t)


In [36]:
#tf 계산
def tf(t, d):
  return 0.5 +0.5*f(t,d)/max([f(w,d) for w in d])

In [37]:
#idf 계산
def idf(t,D):
  numerator = len(D)
  denominator = 1+len([ True for d in D if t in d])
  return log10(numerator/denominator)+1

In [38]:
#tf-idf 계산
def tfidf(t,d,D):
  #print(D)
  print(t)
  #print(d)
  print(tf(t,d))
  print(idf(t,D))
  print(tf(t,d)*idf(t,D))
  print("====")
  return tf(t,d)*idf(t, D)

In [39]:
#공백을 기준으로 토큰화
def tokenizer(d):
  return d.split()

In [43]:
#tfidf 계산
def tfidfScorer(D):
  tokenized_D = [tokenizer(d) for d in D]
  result=[]
  for d in tokenized_D:
    result.append([(t, tfidf(t,d,tokenized_D)) for t in d])
  return result

In [44]:
corpus=[d1,d2]

In [45]:
for i, doc in enumerate(tfidfScorer(corpus)):
  print('============document[%d]=============='%i)
  print(doc)

The
0.75
0.8239087409443188
0.6179315557082391
====
cat
1.0
1.0
1.0
====
sat
0.75
0.8239087409443188
0.6179315557082391
====
on
0.75
0.8239087409443188
0.6179315557082391
====
my
0.75
0.8239087409443188
0.6179315557082391
====
face
0.75
1.0
0.75
====
I
0.75
0.8239087409443188
0.6179315557082391
====
hate
0.75
1.0
0.75
====
a
0.75
0.8239087409443188
0.6179315557082391
====
cat
1.0
1.0
1.0
====
The
0.75
0.8239087409443188
0.6179315557082391
====
dog
1.0
1.0
1.0
====
sat
0.75
0.8239087409443188
0.6179315557082391
====
on
0.75
0.8239087409443188
0.6179315557082391
====
my
0.75
0.8239087409443188
0.6179315557082391
====
bed
0.75
1.0
0.75
====
I
0.75
0.8239087409443188
0.6179315557082391
====
love
0.75
1.0
0.75
====
a
0.75
0.8239087409443188
0.6179315557082391
====
dog
1.0
1.0
1.0
====
============document[0]==============
[('The', 0.6179315557082391), ('cat', 1.0), ('sat', 0.6179315557082391), ('on', 0.6179315557082391), ('my', 0.6179315557082391), ('face', 0.75), ('I', 0.6179315557082391),